In [1]:
%%bash

./clean.sh
curl -sLO https://raw.githubusercontent.com/eficode/wait-for/master/wait-for
chmod u+x wait-for

In [2]:
import os
import subprocess
import webbrowser
import threading 

def run_flask_and_curl(app, cmd):
    stdout = None
    result = subprocess.Popen(
        f'./wait-for localhost:5000 -- {cmd}', 
        shell = True,
        stdout = subprocess.PIPE,
        text = True
    )
    try:
        subprocess.run(
            f'flask run', 
            shell = True, 
            env = {'FLASK_APP': app, 'PATH': os.environ['PATH']},
            timeout = 1.5
        )
    except (KeyboardInterrupt, subprocess.TimeoutExpired):
        pass
    try:
        stdout = result.communicate()[0]
    except KeyboardInterrupt:
        pass
    return stdout

def run_flask_and_brwose(app, url, debug = False):
    threading.Timer(1, lambda: webbrowser.open_new(url)).start()
    try:
        subprocess.run(
            f'flask run', 
            shell = True, 
            env = {'FLASK_APP': app, 'PATH': os.environ['PATH'], 'FLASK_DEBUG': '1' if debug else '0'},
        )
    except KeyboardInterrupt:
        pass

# Flask

Piuttosto che il gigantesco Django, meglio il  *micro* framweork [Flask](https://flask.palletsprojects.com/).

In [3]:
%%writefile app_simple.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return 'Hello, world!' 

Writing app_simple.py


In [4]:
run_flask_and_curl('app_simple.py', 'curl http://localhost:5000/')

'Hello, world!'

In [5]:
run_flask_and_curl('app_simple.py', 'curl http://localhost:5000/te')

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>404 Not Found</title>\n<h1>Not Found</h1>\n<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>\n'

In [6]:
%%writefile app_debug.py

from flask import Flask

X = 0 

app = Flask(__name__)

@app.route('/')
def hello():
    return f'Hello, {1/X}!'

Writing app_debug.py


In [7]:
run_flask_and_brwose('app_debug.py', 'http://localhost:5000/')

In [8]:
run_flask_and_brwose('app_debug.py', 'http://localhost:5000/', debug = True)

In [9]:
%%writefile app_get.py

from flask import Flask, request, escape

app = Flask(__name__)

@app.route('/getme', methods = ['GET'])
def getme():
    name = request.args.get('name', '[default]')
    return f'Your name is: {escape(name)}'

Writing app_get.py


In [10]:
run_flask_and_curl('app_get.py', 'curl -X GET "http://127.0.0.1:5000/getme?name=Pino"')

'Your name is: Pino'

In [11]:
%%writefile app_post.py

from flask import Flask, request, escape

app = Flask(__name__)

@app.route('/postme', methods = ['POST'])
def postme():
    name = request.form.get('name', '[default]')
    return f'Your name is: {escape(name)}'

Writing app_post.py


In [12]:
run_flask_and_curl('app_post.py', 'curl -X POST -d name=Pino http://127.0.0.1:5000/postme')

'Your name is: Pino'

In [13]:
%%writefile app_varroutes.py

from flask import Flask

app = Flask(__name__)

@app.route('/product/<int:post_id>')
def post(post_id):
    return f'Product {post_id}'


Writing app_varroutes.py


In [14]:
run_flask_and_curl('app_varroutes.py', 'curl http://127.0.0.1:5000/product/123')

'Product 123'

In [15]:
! mkdir -p tplapp/templates/

In [16]:
%%writefile tplapp/templates/hello.html

<!doctype html>
<title>Hello from Flask</title>
{% if name %}
  <h1>Hello {{ name }}!</h1>
{% else %}
  <h1>Hello, World!</h1>
{% endif %}

Writing tplapp/templates/hello.html


In [17]:
%%writefile tplapp/__init__.py

from flask import Flask, render_template

app = Flask(__name__)

@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name = None):
    return render_template('hello.html', name = name)

Writing tplapp/__init__.py


In [18]:
run_flask_and_brwose('tplapp', 'http://localhost:5000/hello/World')